# Segmenting and Clustering Neighborhoods in Toronto

In this assignment, we will be scraping postal code data from Wikipedia for the city of Toronto. In part 1, we will then wrangle and clean the data in order to get information about venues in the city's neighborhoods using the __Foursquare API__. Then, in part 2, we will use a _k_-means algorithm to cluster the neighborhoods by most popular venues. Finally, in part 3, we will visualize the clusters on a map of Toronto.

## Part 0 - Load Necessary Libraries

In [117]:
import pandas as pd
import numpy as np

from geopy.geocoders import Nominatim  # convert an address into latitude and longitude values

import requests  # handle requests
from bs4 import BeautifulSoup  # web scraper

from pandas import json_normalize  # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium  # map rendering library

## Part 1 - Web Scraping

Scrape the Wikipedia page using an HTML parser.

In [118]:
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=862527922'

req = requests.get(url)

soup = BeautifulSoup(req.content, 'html.parser')

Find the table in the HTML, and extract the data in each column. Then create a DataFrame to store all the data.

In [127]:
soup_table = soup.find('table')

# function to get the text from a list of HMTL tags
def get_text_map(lst):
    lst = list(map(lambda s: s.get_text(strip=True), lst))
    
    return lst

# get all the column data
data = soup_table.find_all('td')
data = get_text_map(data)

# get the data for each column
p_codes = data[::3]
bghs = data[1::3]
nbhs = data[2::3]

# replace slashes with commas for each neighborhood
nbhs = list(map(lambda s: s.replace(" / ", ", "), nbhs))

toronto_df = pd.DataFrame(zip(p_codes, bghs, nbhs), columns=['PostalCode', 'Borough', 'Neighborhood'])
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Filter out all the rows that have value of 'Not assigned' in the __Borough__ column. Also, if a neighborhood has a value of 'Not assigned', replace it with its borough.

In [128]:
# drop rows with no boroughs
toronto_df = toronto_df[toronto_df.Borough != 'Not assigned']

# confirm that there are no boroughs with the value 'Not assigned'
print(toronto_df['Borough'].value_counts())

Etobicoke           45
North York          38
Scarborough         38
Downtown Toronto    37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Queen's Park         1
Mississauga          1
Name: Borough, dtype: int64


In [129]:
# replace 'Not assigned' neighborhoods with their boroughs
num_na_before = toronto_df[toronto_df['Neighborhood'] == 'Not assigned'].shape[0]

toronto_df['Neighborhood'] = np.where(toronto_df['Neighborhood'] == 'Not assigned', 
                                      toronto_df['Borough'], 
                                      toronto_df['Neighborhood'])

num_na_after = toronto_df[toronto_df['Neighborhood'] == toronto_df['Borough']].shape[0]

# confirm that there are no neighborhoods with the value 'Not assigned'
# and that the count before and after is the same
print("Number of not assigned neighborhoods before:", num_na_before)
print("Number of not assigned neighborhoods after:", num_na_after)
print('\n')
print(toronto_df['Neighborhood'].value_counts())

Number of not assigned neighborhoods before: 1
Number of not assigned neighborhoods after: 1


St. James Town           2
Runnymede                2
Downsview North          1
Woburn                   1
Old Mill North           1
                        ..
Royal York South East    1
Woodbine Heights         1
Willowdale               1
Alderwood                1
Cloverdale               1
Name: Neighborhood, Length: 210, dtype: int64


In [131]:
print("Number of rows in the table:", toronto_df.shape[0])

Number of rows in the table: 212
